In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`

import $ivy.$

In [4]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

## Start

In [5]:
import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._

In [6]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Dataframe API")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/09/01 17:18:20 INFO SparkContext: Running Spark version 3.5.0
24/09/01 17:18:21 INFO SparkContext: OS info Linux, 6.8.0-40-generic, amd64
24/09/01 17:18:21 INFO SparkContext: Java version 11.0.24
24/09/01 17:18:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/01 17:18:21 INFO ResourceUtils: ==============================================================
24/09/01 17:18:21 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/01 17:18:21 INFO ResourceUtils: ==============================================================
24/09/01 17:18:21 INFO SparkContext: Submitted application: Dataframe API
24/09/01 17:18:21 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHea

spark: SparkSession = org.apache.spark.sql.SparkSession@6fff8352
import spark.implicits._

In [7]:
println(s"spark.version == ${spark.version}")

spark.version == 3.5.0


## 1 Создание DataFrame

### createDataFrame

#### From List

In [8]:
val data1 = Seq(("1", "Spark"), ("2", "Scala"), ("3", "Java"))

data1: Seq[(String, String)] = List(
  ("1", "Spark"),
  ("2", "Scala"),
  ("3", "Java")
)

In [9]:
val df1 = spark.createDataFrame(data1)
df1.show

24/09/01 17:18:33 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/09/01 17:18:33 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/DataFrame/spark-warehouse'.
24/09/01 17:18:35 INFO CodeGenerator: Code generated in 179.713495 ms
24/09/01 17:18:35 INFO CodeGenerator: Code generated in 18.241412 ms


+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



df1: DataFrame = [_1: string, _2: string]

In [10]:
val df1WithNames = df1.withColumnsRenamed(Map("_1" -> "StudentID", "_2" -> "Course"))
df1WithNames.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



df1WithNames: DataFrame = [StudentID: string, Course: string]

##### withSchema

In [30]:
val schema1 = StructType(Array(StructField("StudentID", StringType, false),
                              StructField("Course", StringType, true)))

val schema2 = new StructType()
                    .add(StructField("StudentID", StringType, false))
                    .add(StructField("Course", StringType, true))

val schema3 = StructType(
                StructField("StudentID", StringType, false) :: 
                StructField("Course", StringType, true) :: 
                Nil)

schema1.equals(schema2)
schema2.equals(schema3)

schema1: StructType = StructType(
  StructField("StudentID", StringType, false, {}),
  StructField("Course", StringType, true, {})
)
schema2: StructType = StructType(
  StructField("StudentID", StringType, false, {}),
  StructField("Course", StringType, true, {})
)
schema3: StructType = StructType(
  StructField("StudentID", StringType, false, {}),
  StructField("Course", StringType, true, {})
)
res30_3: Boolean = true
res30_4: Boolean = true

In [15]:
import collection.JavaConverters._

val data2 = data1.map(s => Row(s._1, s._2)).asJava
val df2 = spark.createDataFrame(data2, schema2)
df2.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



import collection.JavaConverters._
data2: java.util.List[Row] = SeqWrapper(List([1,Spark], [2,Scala], [3,Java]))
df2: DataFrame = [StudentID: string, Course: string]

#### fromRDD

In [17]:
val rdd1 = spark.sparkContext.parallelize(data1)
val fromRDD1 = spark.createDataFrame(rdd1)
fromRDD1.show

24/09/01 17:22:51 INFO SparkContext: Starting job: show at cell17.sc:3
24/09/01 17:22:51 INFO DAGScheduler: Got job 3 (show at cell17.sc:3) with 1 output partitions
24/09/01 17:22:51 INFO DAGScheduler: Final stage: ResultStage 3 (show at cell17.sc:3)
24/09/01 17:22:51 INFO DAGScheduler: Parents of final stage: List()
24/09/01 17:22:51 INFO DAGScheduler: Missing parents: List()
24/09/01 17:22:51 INFO DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[7] at show at cell17.sc:3), which has no missing parents
24/09/01 17:22:51 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 13.0 KiB, free 4.5 GiB)
24/09/01 17:22:51 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 5.7 KiB, free 4.5 GiB)
24/09/01 17:22:51 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on ubuntu:36981 (size: 5.7 KiB, free: 4.5 GiB)
24/09/01 17:22:51 INFO SparkContext: Created broadcast 3 from broadcast at DAGScheduler.scala:1580
24/09/01 17:22

+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



rdd1: org.apache.spark.rdd.RDD[(String, String)] = ParallelCollectionRDD[4] at parallelize at cell17.sc:1
fromRDD1: DataFrame = [_1: string, _2: string]

##### withSchema

In [19]:
val rdd2 = rdd1.map(s => Row(s._1, s._2))
val fromRDD2 = spark.createDataFrame(rdd2, schema2)
fromRDD2.show

24/09/01 17:23:48 INFO CodeGenerator: Code generated in 9.94827 ms
24/09/01 17:23:48 INFO SparkContext: Starting job: show at cell19.sc:3
24/09/01 17:23:48 INFO DAGScheduler: Got job 6 (show at cell19.sc:3) with 1 output partitions
24/09/01 17:23:48 INFO DAGScheduler: Final stage: ResultStage 6 (show at cell19.sc:3)
24/09/01 17:23:48 INFO DAGScheduler: Parents of final stage: List()
24/09/01 17:23:48 INFO DAGScheduler: Missing parents: List()
24/09/01 17:23:48 INFO DAGScheduler: Submitting ResultStage 6 (MapPartitionsRDD[12] at show at cell19.sc:3), which has no missing parents
24/09/01 17:23:48 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 21.4 KiB, free 4.5 GiB)
24/09/01 17:23:48 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 9.8 KiB, free 4.5 GiB)
24/09/01 17:23:48 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on ubuntu:36981 (size: 9.8 KiB, free: 4.5 GiB)
24/09/01 17:23:48 INFO SparkContext: Created 

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



rdd2: org.apache.spark.rdd.RDD[Row] = MapPartitionsRDD[9] at map at cell19.sc:1
fromRDD2: DataFrame = [StudentID: string, Course: string]

### toDF

#### From List

In [20]:
val df11 = data1.toDF()
df11.show

24/09/01 17:29:36 INFO CodeGenerator: Code generated in 12.90181 ms


+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



df11: DataFrame = [_1: string, _2: string]

In [21]:
val columns = List("StudentID", "Course")
val df12 = data1.toDF(columns: _*)
df12.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



columns: List[String] = List("StudentID", "Course")
df12: DataFrame = [StudentID: string, Course: string]

#### fromRDD

In [23]:
val fromRDD11 = rdd1.toDF()
fromRDD11.show

24/09/01 17:36:44 INFO SparkContext: Starting job: show at cell23.sc:2
24/09/01 17:36:44 INFO DAGScheduler: Got job 12 (show at cell23.sc:2) with 1 output partitions
24/09/01 17:36:44 INFO DAGScheduler: Final stage: ResultStage 12 (show at cell23.sc:2)
24/09/01 17:36:44 INFO DAGScheduler: Parents of final stage: List()
24/09/01 17:36:44 INFO DAGScheduler: Missing parents: List()
24/09/01 17:36:44 INFO DAGScheduler: Submitting ResultStage 12 (MapPartitionsRDD[18] at show at cell23.sc:2), which has no missing parents
24/09/01 17:36:44 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 13.0 KiB, free 4.5 GiB)
24/09/01 17:36:44 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 5.7 KiB, free 4.5 GiB)
24/09/01 17:36:44 INFO BlockManagerInfo: Added broadcast_12_piece0 in memory on ubuntu:36981 (size: 5.7 KiB, free: 4.5 GiB)
24/09/01 17:36:44 INFO SparkContext: Created broadcast 12 from broadcast at DAGScheduler.scala:1580
24/09/

+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



fromRDD11: DataFrame = [_1: string, _2: string]

In [24]:
val fromRDD12 = rdd1.toDF(columns: _*)
fromRDD12.show

24/09/01 17:36:56 INFO SparkContext: Starting job: show at cell24.sc:2
24/09/01 17:36:56 INFO DAGScheduler: Got job 15 (show at cell24.sc:2) with 1 output partitions
24/09/01 17:36:56 INFO DAGScheduler: Final stage: ResultStage 15 (show at cell24.sc:2)
24/09/01 17:36:56 INFO DAGScheduler: Parents of final stage: List()
24/09/01 17:36:56 INFO DAGScheduler: Missing parents: List()
24/09/01 17:36:56 INFO DAGScheduler: Submitting ResultStage 15 (MapPartitionsRDD[21] at show at cell24.sc:2), which has no missing parents
24/09/01 17:36:56 INFO MemoryStore: Block broadcast_15 stored as values in memory (estimated size 13.0 KiB, free 4.5 GiB)
24/09/01 17:36:56 INFO MemoryStore: Block broadcast_15_piece0 stored as bytes in memory (estimated size 5.8 KiB, free 4.5 GiB)
24/09/01 17:36:56 INFO BlockManagerInfo: Added broadcast_15_piece0 in memory on ubuntu:36981 (size: 5.8 KiB, free: 4.5 GiB)
24/09/01 17:36:56 INFO SparkContext: Created broadcast 15 from broadcast at DAGScheduler.scala:1580
24/09/

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



fromRDD12: DataFrame = [StudentID: string, Course: string]

### fromFile

In [28]:
val fromFile1 = spark.read.format("json").load("data/customer_data.json")
fromFile1.show(5, false)

24/09/01 17:40:02 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/09/01 17:40:02 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/09/01 17:40:02 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:40:02 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:40:02 INFO MemoryStore: Block broadcast_22 stored as values in memory (estimated size 198.5 KiB, free 4.5 GiB)
24/09/01 17:40:02 INFO MemoryStore: Block broadcast_22_piece0 stored as bytes in memory (estimated size 33.9 KiB, free 4.5 GiB)
24/09/01 17:40:02 INFO BlockManagerInfo: Added broadcast_22_piece0 in memory on ubuntu:36981 (size: 33.9 KiB, free: 4.5 GiB)
24/09/01 17:40:02 INFO SparkContext: Created broadcast 22 from load at cell28.sc:1
24/09/01 17:40:02 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:40:02 INFO SparkContext: Starting job: load at cell28.sc:1
24/09/01 17:40:02 INFO

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|55711 Janet Plaza Apt. 865\nChristinachester, CT 62716|1988-06-21 00:15:34|Iceland       |12347     |timothy78@hotmail.com   |Katherine David|hillrachel      |
|Unit 2676 Box 9352\nDPO AA 38560                      |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, 

fromFile1: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

In [31]:
val fileSchema = StructType(
                    StructField("Address", StringType, true) ::
                    StructField("Birthdate", StringType, true) ::
                    StructField("Country", StringType, true) ::
                    StructField("CustomerID", StringType, true) ::
                    StructField("Email", StringType, true) ::
                    StructField("Name", StringType, true) ::
                    StructField("Username", StringType, true) ::
                    Nil)

fileSchema: StructType = StructType(
  StructField("Address", StringType, true, {}),
  StructField("Birthdate", StringType, true, {}),
  StructField("Country", StringType, true, {}),
  StructField("CustomerID", StringType, true, {}),
  StructField("Email", StringType, true, {}),
  StructField("Name", StringType, true, {}),
  StructField("Username", StringType, true, {})
)

In [32]:
val fromFile2 = spark.read.format("json").schema(fileSchema).load("data/customer_data.json")
fromFile2.show(5, false)

24/09/01 17:43:57 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/09/01 17:43:57 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:43:57 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:43:57 INFO MemoryStore: Block broadcast_26 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:43:57 INFO MemoryStore: Block broadcast_26_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:43:57 INFO BlockManagerInfo: Added broadcast_26_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:43:57 INFO SparkContext: Created broadcast 26 from show at cell32.sc:2
24/09/01 17:43:57 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:43:57 INFO SparkContext: Starting job: show at cell32.sc:2
24/09/01 17:43:57 INFO DAGScheduler: Got job 22 (show at cell32.sc:2) with 1 output partitions
24/09/01 17:43

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|55711 Janet Plaza Apt. 865\nChristinachester, CT 62716|1988-06-21 00:15:34|Iceland       |12347     |timothy78@hotmail.com   |Katherine David|hillrachel      |
|Unit 2676 Box 9352\nDPO AA 38560                      |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, 

fromFile2: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

## 2 Основные операции

In [33]:
val customerDf = fromFile1

customerDf: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

In [34]:
customerDf.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)



In [36]:
customerDf.dtypes

res36: Array[(String, String)] = Array(
  ("Address", "StringType"),
  ("Birthdate", "StringType"),
  ("Country", "StringType"),
  ("CustomerID", "StringType"),
  ("Email", "StringType"),
  ("Name", "StringType"),
  ("Username", "StringType")
)

In [35]:
customerDf.head

24/09/01 17:51:12 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:51:12 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:51:12 INFO MemoryStore: Block broadcast_28 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:51:12 INFO MemoryStore: Block broadcast_28_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:51:12 INFO BlockManagerInfo: Added broadcast_28_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:51:12 INFO SparkContext: Created broadcast 28 from head at cell35.sc:1
24/09/01 17:51:12 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:51:12 INFO SparkContext: Starting job: head at cell35.sc:1
24/09/01 17:51:12 INFO DAGScheduler: Got job 23 (head at cell35.sc:1) with 1 output partitions
24/09/01 17:51:12 INFO DAGScheduler: Final stage: ResultStage 23 (head at cell35.sc:1)
24/09/01 17:51

res35: Row = [Unit 1047 Box 4089
DPO AA 57348,1994-02-20 00:46:27,United Kingdom,12346,cooperalexis@hotmail.com,Lindsay Cowan,valenciajennifer]

#### Select

In [37]:
customerDf.select("Birthdate", "Country").show

24/09/01 17:51:37 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:51:37 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:51:37 INFO CodeGenerator: Code generated in 7.820498 ms
24/09/01 17:51:37 INFO MemoryStore: Block broadcast_30 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:51:37 INFO BlockManagerInfo: Removed broadcast_28_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:51:37 INFO BlockManagerInfo: Removed broadcast_29_piece0 on ubuntu:36981 in memory (size: 6.0 KiB, free: 4.5 GiB)
24/09/01 17:51:37 INFO MemoryStore: Block broadcast_30_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:51:37 INFO BlockManagerInfo: Added broadcast_30_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:51:37 INFO SparkContext: Created broadcast 30 from show at cell37.sc:1
24/09/01 17:51:37 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 byte

+-------------------+--------------+
|          Birthdate|       Country|
+-------------------+--------------+
|1994-02-20 00:46:27|United Kingdom|
|1988-06-21 00:15:34|       Iceland|
|1974-11-26 15:30:20|       Finland|
|1977-05-06 23:57:35|         Italy|
|1996-09-13 19:14:27|        Norway|
|1969-06-21 03:39:20|        Norway|
|1993-02-25 18:37:29|       Bahrain|
|1993-03-13 12:37:34|         Spain|
|1972-11-10 12:01:08|       Bahrain|
|1973-01-13 17:17:26|      Portugal|
|1989-11-24 17:12:54|   Switzerland|
|1977-06-19 22:35:52|       Austria|
|1983-09-21 05:22:18|        Cyprus|
|1980-10-28 17:25:59|       Austria|
|1982-09-01 09:12:57|       Belgium|
|1979-02-03 03:42:47|       Belgium|
|1974-12-21 13:27:20|   Unspecified|
|1990-07-17 15:47:12|       Belgium|
|1981-07-10 00:35:00|        Cyprus|
|1989-12-26 00:58:01|       Denmark|
+-------------------+--------------+
only showing top 20 rows



In [38]:
customerDf.select(col("Country")).show

24/09/01 17:51:45 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:51:45 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:51:45 INFO CodeGenerator: Code generated in 6.315174 ms
24/09/01 17:51:45 INFO MemoryStore: Block broadcast_32 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:51:45 INFO BlockManagerInfo: Removed broadcast_30_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:51:45 INFO MemoryStore: Block broadcast_32_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:51:45 INFO BlockManagerInfo: Removed broadcast_31_piece0 on ubuntu:36981 in memory (size: 7.1 KiB, free: 4.5 GiB)
24/09/01 17:51:45 INFO BlockManagerInfo: Added broadcast_32_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:51:45 INFO SparkContext: Created broadcast 32 from show at cell38.sc:1
24/09/01 17:51:45 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 byte

+--------------+
|       Country|
+--------------+
|United Kingdom|
|       Iceland|
|       Finland|
|         Italy|
|        Norway|
|        Norway|
|       Bahrain|
|         Spain|
|       Bahrain|
|      Portugal|
|   Switzerland|
|       Austria|
|        Cyprus|
|       Austria|
|       Belgium|
|       Belgium|
|   Unspecified|
|       Belgium|
|        Cyprus|
|       Denmark|
+--------------+
only showing top 20 rows



In [40]:
customerDf.selectExpr("Birthdate as Date").show

24/09/01 17:52:12 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:52:12 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:52:12 INFO MemoryStore: Block broadcast_36 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:52:12 INFO BlockManagerInfo: Removed broadcast_35_piece0 on ubuntu:36981 in memory (size: 7.0 KiB, free: 4.5 GiB)
24/09/01 17:52:12 INFO MemoryStore: Block broadcast_36_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:52:12 INFO BlockManagerInfo: Added broadcast_36_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52:12 INFO SparkContext: Created broadcast 36 from show at cell40.sc:1
24/09/01 17:52:12 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:52:12 INFO BlockManagerInfo: Removed broadcast_34_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52

+-------------------+
|               Date|
+-------------------+
|1994-02-20 00:46:27|
|1988-06-21 00:15:34|
|1974-11-26 15:30:20|
|1977-05-06 23:57:35|
|1996-09-13 19:14:27|
|1969-06-21 03:39:20|
|1993-02-25 18:37:29|
|1993-03-13 12:37:34|
|1972-11-10 12:01:08|
|1973-01-13 17:17:26|
|1989-11-24 17:12:54|
|1977-06-19 22:35:52|
|1983-09-21 05:22:18|
|1980-10-28 17:25:59|
|1982-09-01 09:12:57|
|1979-02-03 03:42:47|
|1974-12-21 13:27:20|
|1990-07-17 15:47:12|
|1981-07-10 00:35:00|
|1989-12-26 00:58:01|
+-------------------+
only showing top 20 rows



In [41]:
customerDf.withColumn("Flag", lit(true)).show

24/09/01 17:52:19 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:52:19 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:52:19 INFO CodeGenerator: Code generated in 11.2263 ms
24/09/01 17:52:19 INFO MemoryStore: Block broadcast_38 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:52:19 INFO BlockManagerInfo: Removed broadcast_36_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52:19 INFO BlockManagerInfo: Removed broadcast_37_piece0 on ubuntu:36981 in memory (size: 7.0 KiB, free: 4.5 GiB)
24/09/01 17:52:19 INFO MemoryStore: Block broadcast_38_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:52:19 INFO BlockManagerInfo: Added broadcast_38_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52:19 INFO SparkContext: Created broadcast 38 from show at cell41.sc:1
24/09/01 17:52:19 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes

+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+----+
|             Address|          Birthdate|       Country|CustomerID|               Email|             Name|        Username|Flag|
+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+----+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|    Lindsay Cowan|valenciajennifer|true|
|55711 Janet Plaza...|1988-06-21 00:15:34|       Iceland|     12347|timothy78@hotmail...|  Katherine David|      hillrachel|true|
|Unit 2676 Box 935...|1974-11-26 15:30:20|       Finland|     12348| tcrawford@gmail.com|  Leslie Martinez|    serranobrian|true|
|2765 Powers Meado...|1977-05-06 23:57:35|         Italy|     12349|  dustin37@yahoo.com|    Brad Cardenas|   charleshudson|true|
|17677 Mark Crest\...|1996-09-13 19:14:27|        Norway|     12350|amyholland@yahoo.com| 

In [42]:
customerDf.withColumnRenamed("Birthdate", "Date").show

24/09/01 17:52:29 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:52:29 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:52:29 INFO MemoryStore: Block broadcast_40 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:52:29 INFO BlockManagerInfo: Removed broadcast_39_piece0 on ubuntu:36981 in memory (size: 7.7 KiB, free: 4.5 GiB)
24/09/01 17:52:29 INFO BlockManagerInfo: Removed broadcast_38_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52:29 INFO MemoryStore: Block broadcast_40_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:52:29 INFO BlockManagerInfo: Added broadcast_40_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52:29 INFO SparkContext: Created broadcast 40 from show at cell42.sc:1
24/09/01 17:52:29 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:52

+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|             Address|               Date|       Country|CustomerID|               Email|             Name|        Username|
+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|    Lindsay Cowan|valenciajennifer|
|55711 Janet Plaza...|1988-06-21 00:15:34|       Iceland|     12347|timothy78@hotmail...|  Katherine David|      hillrachel|
|Unit 2676 Box 935...|1974-11-26 15:30:20|       Finland|     12348| tcrawford@gmail.com|  Leslie Martinez|    serranobrian|
|2765 Powers Meado...|1977-05-06 23:57:35|         Italy|     12349|  dustin37@yahoo.com|    Brad Cardenas|   charleshudson|
|17677 Mark Crest\...|1996-09-13 19:14:27|        Norway|     12350|amyholland@yahoo.com|     Natalie Ford| gregoryharrison|


#### Filter

In [43]:
customerDf.filter("Country = 'Norway'").show

24/09/01 17:52:37 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Country),EqualTo(Country,Norway)
24/09/01 17:52:37 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Country#254),(Country#254 = Norway)
24/09/01 17:52:37 INFO CodeGenerator: Code generated in 11.102797 ms
24/09/01 17:52:37 INFO MemoryStore: Block broadcast_42 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:52:37 INFO MemoryStore: Block broadcast_42_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:52:37 INFO BlockManagerInfo: Removed broadcast_40_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52:37 INFO BlockManagerInfo: Added broadcast_42_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52:37 INFO SparkContext: Created broadcast 42 from show at cell43.sc:1
24/09/01 17:52:37 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning

+--------------------+-------------------+-------+----------+--------------------+----------------+---------------+
|             Address|          Birthdate|Country|CustomerID|               Email|            Name|       Username|
+--------------------+-------------------+-------+----------+--------------------+----------------+---------------+
|17677 Mark Crest\...|1996-09-13 19:14:27| Norway|     12350|amyholland@yahoo.com|    Natalie Ford|gregoryharrison|
|50047 Smith Point...|1969-06-21 03:39:20| Norway|     12352| vcarter@hotmail.com|     Dana Clarke|         hmyers|
|0297 Jacob Ranch ...|1990-06-01 14:49:52| Norway|     12381|douglaschavez@hot...|   Matthew Jones|    stephanie68|
|3102 Hopkins Walk...|1976-06-19 08:10:24| Norway|     12430|crystalromero@hot...|       Lisa Tate|       pgilbert|
|637 Philip Lock S...|1984-06-06 09:36:14| Norway|     12432|jessica87@hotmail...|  Cheryl Herring|mathewsnicholas|
|546 Tyler Prairie...|1985-05-27 10:39:47| Norway|     12433|mariahmcphe

In [56]:
customerDf.where($"Country" =!= "Iceland").show(false)

24/09/01 21:56:01 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Country),Not(EqualTo(Country,Iceland))
24/09/01 21:56:01 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Country#254),NOT (Country#254 = Iceland)
24/09/01 21:56:01 INFO MemoryStore: Block broadcast_65 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 21:56:01 INFO MemoryStore: Block broadcast_65_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 21:56:01 INFO BlockManagerInfo: Added broadcast_65_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 21:56:01 INFO SparkContext: Created broadcast 65 from show at cell56.sc:1
24/09/01 21:56:01 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 21:56:01 INFO SparkContext: Starting job: show at cell56.sc:1
24/09/01 21:56:01 INFO DAGScheduler: Got job 41 (show at cell56.sc:1) with 1 output partitions
24/

+-----------------------------------------------------+-------------------+--------------+----------+--------------------------+-----------------+----------------+
|Address                                              |Birthdate          |Country       |CustomerID|Email                     |Name             |Username        |
+-----------------------------------------------------+-------------------+--------------+----------+--------------------------+-----------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                     |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com  |Lindsay Cowan    |valenciajennifer|
|Unit 2676 Box 9352\nDPO AA 38560                     |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com       |Leslie Martinez  |serranobrian    |
|2765 Powers Meadow\nHeatherfurt, CT 53165            |1977-05-06 23:57:35|Italy         |12349     |dustin37@yahoo.com        |Brad Cardenas    |charleshudson   |
|17677 Mark Cres

#### Sort

In [57]:
customerDf.sort(col("CustomerID").desc).show

24/09/01 21:56:34 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 21:56:34 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 21:56:34 INFO MemoryStore: Block broadcast_67 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 21:56:34 INFO MemoryStore: Block broadcast_67_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 21:56:34 INFO BlockManagerInfo: Added broadcast_67_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 21:56:34 INFO SparkContext: Created broadcast 67 from show at cell57.sc:1
24/09/01 21:56:34 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 21:56:34 INFO SparkContext: Starting job: show at cell57.sc:1
24/09/01 21:56:34 INFO DAGScheduler: Got job 42 (show at cell57.sc:1) with 1 output partitions
24/09/01 21:56:34 INFO DAGScheduler: Final stage: ResultStage 42 (show at cell57.sc:1)
24/09/01 21:56

+--------------------+-------------------+--------------+----------+--------------------+--------------------+-----------------+
|             Address|          Birthdate|       Country|CustomerID|               Email|                Name|         Username|
+--------------------+-------------------+--------------+----------+--------------------+--------------------+-----------------+
|6942 Connie Skywa...|1973-10-24 00:52:10|United Kingdom|     12989| amber97@hotmail.com|   Brandon Contreras|           ecasey|
|79375 David Neck\...|1971-05-04 22:20:10|United Kingdom|     12988|   erica98@gmail.com|      Gabriel Romero|          qknight|
|00881 West Flat\n...|1997-03-05 19:20:57|United Kingdom|     12987|    vkeith@yahoo.com|Christopher Lawrence|        smcintyre|
|499 Jonathan Stre...|1987-10-24 20:05:15|United Kingdom|     12985| fredsmith@yahoo.com|        Xavier Myers|stricklandjeffery|
|9505 Melissa Stre...|1975-09-22 15:21:58|United Kingdom|     12984|scottjonathan@yah...|        

In [46]:
customerDf.orderBy("CustomerID").show

24/09/01 17:52:53 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:52:53 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:52:53 INFO CodeGenerator: Code generated in 5.601076 ms
24/09/01 17:52:53 INFO MemoryStore: Block broadcast_48 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:52:53 INFO BlockManagerInfo: Removed broadcast_47_piece0 on ubuntu:36981 in memory (size: 7.0 KiB, free: 4.5 GiB)
24/09/01 17:52:53 INFO MemoryStore: Block broadcast_48_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:52:53 INFO BlockManagerInfo: Added broadcast_48_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52:53 INFO SparkContext: Created broadcast 48 from show at cell46.sc:1
24/09/01 17:52:53 INFO BlockManagerInfo: Removed broadcast_46_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:52:53 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 byte

+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|             Address|          Birthdate|       Country|CustomerID|               Email|             Name|        Username|
+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|    Lindsay Cowan|valenciajennifer|
|55711 Janet Plaza...|1988-06-21 00:15:34|       Iceland|     12347|timothy78@hotmail...|  Katherine David|      hillrachel|
|Unit 2676 Box 935...|1974-11-26 15:30:20|       Finland|     12348| tcrawford@gmail.com|  Leslie Martinez|    serranobrian|
|2765 Powers Meado...|1977-05-06 23:57:35|         Italy|     12349|  dustin37@yahoo.com|    Brad Cardenas|   charleshudson|
|17677 Mark Crest\...|1996-09-13 19:14:27|        Norway|     12350|amyholland@yahoo.com|     Natalie Ford| gregoryharrison|


#### Repartition

In [49]:
val repartitionedDf = customerDf.repartition(5, col("Country"))
println(s"Old num partitions: ${customerDf.rdd.getNumPartitions}")
println(s"New num partitions: ${repartitionedDf.rdd.getNumPartitions}")

24/09/01 17:53:18 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:53:18 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:53:18 INFO MemoryStore: Block broadcast_53 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:53:18 INFO MemoryStore: Block broadcast_53_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:53:18 INFO BlockManagerInfo: Added broadcast_53_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:53:18 INFO SparkContext: Created broadcast 53 from rdd at cell49.sc:2
24/09/01 17:53:18 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:53:18 INFO DAGScheduler: Registering RDD 100 (rdd at cell49.sc:2) as input to shuffle 1
24/09/01 17:53:18 INFO DAGScheduler: Got map stage job 35 (rdd at cell49.sc:2) with 1 output partitions
24/09/01 17:53:18 INFO DAGScheduler: Final stage: ShuffleMapStag

New num partitions: 5


repartitionedDf: Dataset[Row] = [Address: string, Birthdate: string ... 5 more fields]

In [50]:
println(s"Num partitions after coalesce: ${repartitionedDf.coalesce(1).rdd.getNumPartitions}")

24/09/01 17:54:06 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:54:06 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:54:06 INFO MemoryStore: Block broadcast_55 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:54:06 INFO BlockManagerInfo: Removed broadcast_54_piece0 on ubuntu:36981 in memory (size: 7.3 KiB, free: 4.5 GiB)
24/09/01 17:54:06 INFO MemoryStore: Block broadcast_55_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:54:06 INFO BlockManagerInfo: Added broadcast_55_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:54:06 INFO SparkContext: Created broadcast 55 from rdd at cell50.sc:1
24/09/01 17:54:06 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:54:06 INFO DAGScheduler: Registering RDD 107 (rdd at cell50.sc:1) as input to shuffle 2
24/09/01 17:54:06 INFO DAGScheduler: Got m

Num partitions after coalesce: 1


### functions

In [51]:
customerDf.select("Birthdate").show(5)

24/09/01 17:54:13 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:54:13 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:54:13 INFO MemoryStore: Block broadcast_57 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:54:13 INFO BlockManagerInfo: Removed broadcast_56_piece0 on ubuntu:36981 in memory (size: 7.3 KiB, free: 4.5 GiB)
24/09/01 17:54:13 INFO BlockManagerInfo: Removed broadcast_55_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:54:13 INFO MemoryStore: Block broadcast_57_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:54:13 INFO BlockManagerInfo: Added broadcast_57_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:54:13 INFO SparkContext: Created broadcast 57 from show at cell51.sc:1
24/09/01 17:54:13 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:54

+-------------------+
|          Birthdate|
+-------------------+
|1994-02-20 00:46:27|
|1988-06-21 00:15:34|
|1974-11-26 15:30:20|
|1977-05-06 23:57:35|
|1996-09-13 19:14:27|
+-------------------+
only showing top 5 rows



In [52]:
customerDf.select(date_format(col("Birthdate"), "yyyy-MM-dd").alias("bd")).show(5)

24/09/01 17:54:26 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:54:26 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:54:26 INFO CodeGenerator: Code generated in 14.798035 ms
24/09/01 17:54:26 INFO MemoryStore: Block broadcast_59 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:54:26 INFO BlockManagerInfo: Removed broadcast_58_piece0 on ubuntu:36981 in memory (size: 7.0 KiB, free: 4.5 GiB)
24/09/01 17:54:26 INFO MemoryStore: Block broadcast_59_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:54:26 INFO BlockManagerInfo: Added broadcast_59_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:54:26 INFO SparkContext: Created broadcast 59 from show at cell52.sc:1
24/09/01 17:54:26 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:54:26 INFO BlockManagerInfo: Removed broadcast_57_piece0 on 

+----------+
|        bd|
+----------+
|1994-02-20|
|1988-06-21|
|1974-11-26|
|1977-05-06|
|1996-09-13|
+----------+
only showing top 5 rows



In [53]:
customerDf.select("Name", "Username").show(5)

24/09/01 17:54:32 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:54:32 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:54:32 INFO MemoryStore: Block broadcast_61 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:54:32 INFO BlockManagerInfo: Removed broadcast_59_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:54:32 INFO MemoryStore: Block broadcast_61_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:54:32 INFO BlockManagerInfo: Added broadcast_61_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:54:32 INFO BlockManagerInfo: Removed broadcast_60_piece0 on ubuntu:36981 in memory (size: 7.6 KiB, free: 4.5 GiB)
24/09/01 17:54:32 INFO SparkContext: Created broadcast 61 from show at cell53.sc:1
24/09/01 17:54:32 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/01 17:54

+---------------+----------------+
|           Name|        Username|
+---------------+----------------+
|  Lindsay Cowan|valenciajennifer|
|Katherine David|      hillrachel|
|Leslie Martinez|    serranobrian|
|  Brad Cardenas|   charleshudson|
|   Natalie Ford| gregoryharrison|
+---------------+----------------+
only showing top 5 rows



In [54]:
customerDf.withColumn("Identity", array('Name, 'Username)).printSchema

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- Identity: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [55]:
customerDf
    .withColumn("Identity", array('Name, 'Username))
    .select("Name", "Username", "Identity")
    .show(5, false)

24/09/01 17:54:55 INFO FileSourceStrategy: Pushed Filters: 
24/09/01 17:54:55 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/01 17:54:55 INFO CodeGenerator: Code generated in 9.001022 ms
24/09/01 17:54:55 INFO MemoryStore: Block broadcast_63 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/01 17:54:55 INFO BlockManagerInfo: Removed broadcast_62_piece0 on ubuntu:36981 in memory (size: 7.1 KiB, free: 4.5 GiB)
24/09/01 17:54:55 INFO BlockManagerInfo: Removed broadcast_61_piece0 on ubuntu:36981 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:54:55 INFO MemoryStore: Block broadcast_63_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/01 17:54:55 INFO BlockManagerInfo: Added broadcast_63_piece0 in memory on ubuntu:36981 (size: 34.0 KiB, free: 4.5 GiB)
24/09/01 17:54:55 INFO SparkContext: Created broadcast 63 from show at cell55.sc:4
24/09/01 17:54:55 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 byte

+---------------+----------------+---------------------------------+
|Name           |Username        |Identity                         |
+---------------+----------------+---------------------------------+
|Lindsay Cowan  |valenciajennifer|[Lindsay Cowan, valenciajennifer]|
|Katherine David|hillrachel      |[Katherine David, hillrachel]    |
|Leslie Martinez|serranobrian    |[Leslie Martinez, serranobrian]  |
|Brad Cardenas  |charleshudson   |[Brad Cardenas, charleshudson]   |
|Natalie Ford   |gregoryharrison |[Natalie Ford, gregoryharrison]  |
+---------------+----------------+---------------------------------+
only showing top 5 rows



## 3 Агрегирование

In [ ]:
customerDf.groupBy("Country").agg(count(lit(1))).show

In [ ]:
customerDf.groupBy("Country").count().show

In [ ]:
customerDf.groupBy("Country", "Birthdate").agg(min("CustomerID"), max("CustomerID")).orderBy("Country").show

### union

In [ ]:
customerDf.count

In [ ]:
customerDf.union(customerDf).count

In [ ]:
customerDf.count * 5

In [ ]:
(1 to 5).toList.map(a => customerDf).reduce((x, y) => x.union(y)).count

### join

In [ ]:
val retailDf = spark.read.format("json").load("data/retail_data.json")

In [ ]:
customerDf.printSchema

In [ ]:
retailDf.printSchema

In [ ]:
(customerDf.dtypes.map(_._1)).toSet.intersect((retailDf.dtypes.map(_._1)).toSet)

In [ ]:
customerDf.join(retailDf, "CustomerID").show

In [ ]:
customerDf
    .join(retailDf, customerDf("CustomerID") === retailDf("CustomerID"), "left")
    //.select("CustomerID")
    .show

## Stop

In [ ]:
spark.stop()